# 株価の時系列データの分析

pandas.DataFrame.rolling
- ma: Moving Average
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html<br>

pandas.DataFrame.ewm
- ewm: Exponential Weighted Moving averate
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import mplfinance as mpf

#import japanize_matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#### 指数移動平均の計算
y1は単純な$(1-\alpha^{i})$, y2は$\alpha=2/(5+1)$, y3は$\alpha=2/(25+1)$を採用

In [ ]:
alp, n = 0.1, 30
y1 = [(1-alp)**i for i in range(n)]
alp2 = 2/(5+1)
y2 = [(1-alp2)**i for i in range(n)]
alp3 = 2/(25+1)
y3 = [(1-alp3)**i for i in range(n)]


fig, ax = plt.subplots(figsize=(5,3))
ax.plot(y1, marker='o', color='k', label='alp=0.1')
ax.plot(y2, marker='x', color='k', label='alp=2/(5+1)')
ax.plot(y3, marker='.', color='k', label='alp=2/(25+1)')
#ax.set_facecolor((1,1,1,0))
ax.legend()
ax.grid(linestyle='dotted', color='k')
#plt.savefig('fig_TSA_StockPrice_ExmExponent.pdf', bbox_inches='tight')
plt.show()

## 株価データの読み込み

In [ ]:
df = pd.read_csv('dataStockChart.csv', index_col='Date', parse_dates=[0], comment='#' )
print(df.head())
#print(df[50:].head())

## 移動平均
pandasは次のように称している<br>
- 単純平均(SMA:simple moving average)$\rightarrow$rolling 
- 指数移動平均(EMA: exponential moving average)$\rightarrow$EWM: Exponentially Weighted Moving average

In [ ]:
item='Close' 

sma5  = df[item].rolling(window=5).mean()
sma25 = df[item].rolling(window=25).mean()
ema5  = df[item].ewm(span=5).mean() 
ema25 = df[item].ewm(span=25).mean() 

fig, ax = plt.subplots(figsize=(6,3))
df[item].plot(ax=ax, color='black', label=item, grid=True) # data
sma5.plot(ax=ax,  label='SMA 5', color='dimgray', linestyle='dashed')
sma25.plot(ax=ax, label='SMA 25', color='grey', linestyle='dashed')
ema5.plot(ax=ax,  label='EMA W=5', color='blue', linestyle='dashed')
ema25.plot(ax=ax, label='EMA W=25', color='red', linestyle='dashed')


#ax.set_facecolor((1,1,1,0))
ax.grid(linestyle='dotted', color='k')
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
#plt.savefig('fig_TSA_StockPrice_MA_1.pdf', bbox_inches='tight')
plt.show()

## ボリンジャーバンド（Bollinger bands)
- https://en.wikipedia.org/wiki/Bollinger_Bands

In [ ]:
item = 'Close'
W = 25
sm_mean  = df[item].rolling(window=W).mean()
sm_std = df[item].rolling(window=W).std() #unbiased
m = 2
upper_band = sm_mean + sm_std*m
lower_band = sm_mean - sm_std*m


fig, ax = plt.subplots(figsize=(6,3))
df[item].plot(ax=ax, label=item, color='black', linestyle='solid')
sm_mean.plot(ax=ax, label='SMA 5', color='k', linestyle='dotted')
ax.fill_between(df.index, upper_band, lower_band, color='gray', alpha=0.3)

plt.grid()
plt.legend(loc='upper right')

#plt.savefig('fig_TSA_StockPrice_BollingerBand.pdf', bbox_inches='tight')
plt.show()

## ローソク足チャート
https://en.wikipedia.org/wiki/Candlestick_chart

import mplfinance as mpf
- https://pypi.org/project/mplfinance/
- https://github.com/matplotlib/mplfinance

#### 引数
- type: グラフの種類
    - see site:https://github.com/matplotlib/mplfinance/ 内で"mplfinance" "type"を検索
- style: チャートの見た目（表現），'classic', 'charles', 'brasil', 'yahoo', 'binance'等がある。
    - see https://github.com/matplotlib/mplfinance/　内で“Mplfinance Styles”を検索

In [ ]:
fname = 'fig_TSA_StockPrice_CandleStick_Volume.pdf'
# 画像出力しない場合
mpf.plot(df[24:], type='candle', volume=True,  mav=(20, 50), style='binance', figsize=(9,4))
#mpf.plot(df[24:], type='candle', volume=True,  mav=(20, 50), style='classic', figsize=(9,4))

# 画像出力する場合
#mpf.plot(df[24:], type='candle', volume=True,  mav=(20, 50), style='yahoo', figsize=(9,4), savefig=fname)
plt.close()

### 備考　nullデータの対処
- 株価データにはnullデータ（マーケットが閉じている日は取引が無い）がある場合がある<br>
- pandasはnull（データは無いという意味）をNan(Not a Number, 数字データは無いという意味)と解釈する。
#### この対処法を示す
- 原データを格納する変数（データフレーム）をdf0とする<br>
- メッソド .notnull() はNan(null)データが無い行を返す。
- データが無い日は，Open, High, Low, Close　全て同じであるから，ここでは'Open'を用いる

df0 = pd.read_csv('filename.csv', index_col='Date', parse_dates=[0] )  
df0 = df0.sort_index() # 日付を昇順に並び替える  
df = df0[df0['Open'].notnull()]  
df.head()  